## Advanced Lane Finding Project

The goals / steps of this project are the following:

* 1. Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* 2. Apply a distortion correction to raw images.
* 3. Use color transforms, gradients, etc., to create a thresholded binary image.
* 4. Apply a perspective transform to rectify binary image ("birds-eye view").
* 5. Detect lane pixels and fit to find the lane boundary.
* 6. Determine the curvature of the lane and vehicle position with respect to center.
* 7. Warp the detected lane boundaries back onto the original image.
* 8. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

## 0. Import Modules & My function

In [8]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib qt

# Sobel threshold function (direction = 'x' or 'y')
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))

    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    return grad_binary

# Sobel magnitude threshold function (mag = sqrt(x^2 + y^2))
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    magnitude_sobel = np.sqrt(sobelx**2 + sobely**2)
    scaled_sobel = np.uint8(255*magnitude_sobel/np.max(magnitude_sobel))

    mag_binary = np.zeros_like(scaled_sobel)
    mag_binary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    
    return mag_binary

# Direction threshold function (direction angle = artan2(y, x))
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))

    dir_binary = np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    
    return dir_binary

# HSV color threshold function
def hsv_color_threshold(img, h_thresh=(130,255), s_thresh=(130,255), v_thresh=(130,255)):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h_channel = hsv[:,:,0]
    s_channel = hsv[:,:,1]
    v_channel = hsv[:,:,2]
    
    # Threshold color channel
    h_binary = np.zeros_like(h_channel)
    h_binary[(h_channel >= h_thresh[0]) & (h_channel <= h_thresh[1])] = 1
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >= v_thresh[0]) & (v_channel <= v_thresh[1])] = 1
    
    return h_binary, s_binary, v_binary

# HLS color threshold function
def hls_color_threshold(img, h_thresh=(130,255), l_thresh=(130,255), s_thresh=(130,255)):
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Threshold color channel
    h_binary = np.zeros_like(h_channel)
    h_binary[(h_channel >= h_thresh[0]) & (h_channel <= h_thresh[1])] = 1
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
    
    return h_binary, l_binary, s_binary

# LAB color threshold function
def lab_color_threshold(img, l_thresh=(130,255), a_thresh=(130,255), b_thresh=(130,255)):
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    lab = cv2.medianBlur(lab, 5)
    l_channel = lab[:,:,0]
    a_channel = lab[:,:,1]
    b_channel = lab[:,:,2]
    
    # Threshold color channel
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
    a_binary = np.zeros_like(a_channel)
    a_binary[(a_channel >= a_thresh[0]) & (a_channel <= a_thresh[1])] = 1
    b_binary = np.zeros_like(b_channel)
    b_binary[(b_channel >= b_thresh[0]) & (b_channel <= b_thresh[1])] = 1
    
    return l_binary, a_binary, b_binary

# binary threshold function (In this project, I used Sobelx, hls_s_channel and hsv_v_channel)
def binary_threshold(img):
    ksize = 15
    
    # Take gardient threshold value
    grad_x_binary = abs_sobel_thresh(img, orient='x', sobel_kernel=ksize, thresh=(20, 255))
    grad_y_binary = abs_sobel_thresh(img, orient='y', sobel_kernel=ksize, thresh=(20, 255))
    mag_binary = mag_thresh(img, sobel_kernel=ksize, mag_thresh=(30, 100))
    dir_binary = dir_threshold(img, sobel_kernel=ksize, thresh=(0.7, 1.3))
    
    # Take color threshold value
    hls_h_binary, hls_l_binary, hls_s_binary = hls_color_threshold(img,(130,255),(100,255),(170,255))
    hsv_h_binary, hsv_s_binary, hsv_v_binary = hsv_color_threshold(img,(130,255),(130,255),(170,255)) #200
    lab_l_binary, lab_a_binary, lab_b_binary = lab_color_threshold(img,(130,255),(130,255),(130,255))
    
    combined = np.zeros_like(hls_s_binary)

    # ('sobel x' or 'hls_s_bin') and 'hsv_v_bin' <- Overlap condition
    combined[((grad_x_binary == 1) | (hls_s_binary == 1)) & (hsv_v_binary == 1)] = 1
    
    return combined

# Perspective transform function
def warpImage(img, img_size, unwarp = False):
    # get 'src' and 'dst' value. (according to img_size, region is changed)
    # take polygon area (ROI)
    src = np.float32(
       [[(img_size[0]/2)-65, (img_size[1]/2)+100],
        [(img_size[0]/2)+65, (img_size[1]/2)+100],
        [(img_size[0]*5/6)+60, img_size[1]],
        [(img_size[0]/6)-60, img_size[1]]])
    
    dst = np.float32(
       [[(img_size[0]/4), 0],
        [(img_size[0]*3/4), 0],
        [(img_size[0]*3/4), img_size[1]],
        [(img_size[0]/4), img_size[1]]])
    
    # Using perspective transform function in opencv
    # To back onto original image, 'unwarp = True' and change parameter 'src' and 'dst'
    M = cv2.getPerspectiveTransform(src, dst)
    revM = cv2.getPerspectiveTransform(dst, src)
    if unwarp == False :
        # warp the image
        warped_img = cv2.warpPerspective(img, M, img_size)
    else :
        warped_img = cv2.warpPerspective(img, revM, img_size)
    
    return src, warped_img

# Polynomial fitting function (out_img is maked as seperately left and right lines)
def fit_polynomial(img, binary_warped):
    
    # global variable for polynomial fitting
    global prior_find
    global first_time
    global left_fit
    global right_fit
        
    # Find our lane pixels first
    if first_time == True:
        leftx, lefty, rightx, righty, out_img  = find_lane_pixels(binary_warped)
    else:
        leftx, lefty, rightx, righty, out_img = prior_search(binary_warped, left_fit, right_fit)
        if ((len(leftx) == 0)|(len(rightx)==0)|(len(lefty)==0)|(len(righty)==0)):
            prior_find = False
        if prior_find == False:
            leftx, lefty, rightx, righty, out_img  = find_lane_pixels(binary_warped)
    
    # Using polyfit function, take coefficient of polynominal
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        prior_find = True
        first_time = False
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        prior_find = False

    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    return left_fit, right_fit, left_fitx, right_fitx, ploty, out_img

def prior_search(binary_warped, left_fit, right_fit):

    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))

    left_lane_inds = ((nonzerox > ((left_fit[0]*nonzeroy**2 + left_fit[1]*nonzeroy + left_fit[2])-margin)) &
                      (nonzerox < ((left_fit[0]*nonzeroy**2 + left_fit[1]*nonzeroy + left_fit[2])+margin)))
    right_lane_inds = ((nonzerox > ((right_fit[0]*nonzeroy**2 + right_fit[1]*nonzeroy + right_fit[2])-margin)) &
                       (nonzerox < ((right_fit[0]*nonzeroy**2 + right_fit[1]*nonzeroy + right_fit[2])+margin)))

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img
    
    

# Fining lane pixels function (window search and prior search)
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 70
    # Set minimum number of pixels found to recenter window
    minpix = 40

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low), (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low), (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def complete_img(img, mtx, dist):
    img_size = (img.shape[1], img.shape[0])
    
    # Distortion Correction to 'img'
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    # To take binary images, I use binary_threshold() that was written above section.
    binary_img = binary_threshold(undist)
    
    # For perspective transform, I add the warpImage() function
    src, warped_img = warpImage(binary_img, img_size)
    
    left_fit, right_fit, left_fitx, right_fitx, ploty, out_img = fit_polynomial(img, warped_img)
        
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    y_eval = np.max(ploty)
    center_fit = np.mean([left_fit,right_fit],axis =0)
    center_fitx = np.mean([left_fitx,right_fitx],axis =0)

    left_curverad = ((1+(2*left_fit[0]*y_eval*ym_per_pix+left_fit[1])**2)**(3/2))/np.absolute(2*left_fit[0])  ## Implement the calculation of the left line here
    right_curverad = ((1+(2*right_fit[0]*y_eval*ym_per_pix+right_fit[1])**2)**(3/2))/np.absolute(2*right_fit[0])  ## Implement the calculation of the right line here
    
    unwarp_img = np.zeros((warped_img.shape[0], warped_img.shape[1],3), dtype=np.uint8)
    
    left_lane = np.dstack([left_fitx, ploty])
    right_lane = np.dstack([right_fitx, ploty])
    center_lane = np.dstack([center_fitx, ploty])
    cv2.polylines(unwarp_img, np.int32([left_lane]), False, (255, 0, 0),thickness=40)
    cv2.polylines(unwarp_img, np.int32([right_lane]), False, (0, 0, 255),thickness=40 )
    
    rev_right_lane = np.dstack([np.flip(right_fitx), np.flip(ploty)])
    lane_polygon = np.hstack((left_lane, rev_right_lane))
    
    cv2.fillPoly(unwarp_img, np.int32(lane_polygon), (0, 255, 0))
    cv2.polylines(unwarp_img, np.int32([center_lane]), False, (100, 0, 255),thickness=10)
    center_cruverad = ((1+(2*center_fit[0]*y_eval*ym_per_pix+center_fit[1])**2)**(3/2))/np.absolute(2*center_fit[0])  ## Implement the calculation of the left line here
    center_cruverad = np.int32(center_cruverad)
    
    center_pointx = center_fit[0]*y_eval**2 + center_fit[1]*y_eval + center_fit[2]
    offset = round((center_pointx - img_size[0]/2)*xm_per_pix, 2)
    
    curv_str1 = 'Radius of Curvature = ' + str(center_cruverad) + 'm'
    curv_str2 = 'Vehicle is '+str(offset)+'m'+' left of center'
    curv_str3 = 'left side (+) / right side (-)'
    
    # Warp reversely the image that include drawing lane lines
    src, unwarp_img = warpImage(unwarp_img, img_size, True)
    
    add_img = cv2.addWeighted(undist, 1, unwarp_img, 0.3, 0)
    
    cv2.putText(add_img, curv_str1, (50,40), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(add_img, curv_str2, (50,85), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(add_img, curv_str3, (50,125), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (255,255,255), 2, cv2.LINE_AA)
    
    return add_img

## 1. Camera Callibration (for chessboard)
* Undistort and Perspective Transform
* Input images PATH = /camera_cal
* ouput images PATH = /output_images/01_camera_cal
* The file names of the input and the output image were made to match
* If corners is not found, perspective transform of image will not take

In [3]:
# chessboard size = 6*9
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Get objpoints and imgpoints to apply calibrateCamera() class
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray,(9,6),None)
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        img_size = (img.shape[1], img.shape[0])

# Camera Calibration using objpoints and imgpoints
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
offset = 100
nx = 9
ny = 6
# Saving calibration parameter by using pickle
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "calibration.p", "wb" ) )

# Undistortion and Perspective transform of chessboard image
# If corner is not found, only undistort() class will use
# because perspective transform need corners points
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray,(9,6),None)
    # If found, undistort and warp image
    if ret == True:
        # img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        undist = cv2.undistort(img, mtx, dist, None, mtx)

        src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
        dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                                     [img_size[0]-offset, img_size[1]-offset], 
                                     [offset, img_size[1]-offset]])

        M = cv2.getPerspectiveTransform(src, dst)
        undist = cv2.warpPerspective(undist, M, img_size)
    # If not found, only undistort image not perspective transform
    else:
        undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    filename = os.path.basename(fname)
    cv2.imwrite('output_images/01_camera_cal/' + filename, undist)
    cv2.imshow('warped', undist)
    cv2.waitKey(500)
        
cv2.destroyAllWindows()

## 2. Distortion correction to raw images
* In this pipeline, raw images(/test_images) will be undistorted
* Input images PATH = /test_images
* Output images PATH = /output_images/02_undistort_images

In [15]:
# Load calibration parameter(mtx, dist) from pickle file
dist_pickle = pickle.load( open( "calibration.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

# Load imgae file using glob()
images = glob.glob('test_images/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Distortion Correction to 'img'
    undist = cv2.undistort(img, mtx, dist, None, mtx)

    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(undist)
    ax2.set_title('Undistorted Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
    filename = os.path.basename(fname)
    
    # save the images
    plt.savefig('output_images/02_undistort_images/'+filename)

## 3. Create binary image
* Using color & gradient method
* Input images PATH = /test_images
* Output Images PATH = /output_images/03_binary_images

In [29]:
images = glob.glob('test_images/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    comined_binary = binary_threshold(img)
    
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()

    ax1.imshow(img)
    ax1.set_title('Original image', fontsize=40)

    ax2.imshow(comined_binary, cmap='gray')
    ax2.set_title('Combined Binary', fontsize=40)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

    filename = os.path.basename(fname)
    
    # save the images
    plt.savefig('output_images/03_binary_images/'+filename)


In [59]:

# ############################## Thresh hold test 


# images = glob.glob('test_images/*.jpg')

# for fname in images:
#     img = cv2.imread(fname)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     img_size = (img.shape[1], img.shape[0])
#     src, warp_img = warpImage(img, img_size)
#     ksize = 15
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    
#     grad_x_binary = abs_sobel_thresh(img, orient='x', sobel_kernel=ksize, thresh=(20, 255))
# #     grad_x_binary = cv2.morphologyEx(grad_x_binary, cv2.MORPH_OPEN, kernel)
#     grad_y_binary = abs_sobel_thresh(img, orient='y', sobel_kernel=ksize, thresh=(20, 255))
#     mag_binary = mag_thresh(img, sobel_kernel=ksize, mag_thresh=(30, 100))
#     dir_binary = dir_threshold(img, sobel_kernel=ksize, thresh=(0.7, 1.3))

    
#     hls_h_binary, hls_l_binary, hls_s_binary = hls_color_threshold(img,(130,255),(100,255),(170,255))
#     hsv_h_binary, hsv_s_binary, hsv_v_binary = hsv_color_threshold(img,(130,255),(130,255),(200,255))
#     lab_l_binary, lab_a_binary, lab_b_binary = lab_color_threshold(img,(100,255),(130,255),(130,255))
    
#     combined = np.zeros_like(hls_s_binary)
# #     combined[((grad_x_binary == 1) | (hls_s_binary == 1)) & (hsv_v_binary == 1)] = 1

#     combined[((grad_x_binary == 1) | ((hls_s_binary ==1) &(hsv_v_binary == 1)))] = 1
    
#     # Plot the result
#     f, ([ax1, ax2, ax3], [ax4, ax5, ax6]) = plt.subplots(2, 3, figsize=(24, 9))
#     f.tight_layout()

#     ax1.imshow(grad_x_binary, cmap='gray')
#     ax1.set_title('grad_x_binary', fontsize=40)
    
#     ax2.imshow(hls_s_binary, cmap='gray')
#     ax2.set_title('hls_s_binary', fontsize=40)

#     ax3.imshow(hsv_v_binary, cmap='gray')
#     ax3.set_title('hsv_v_binary', fontsize=40)
    
#     ax4.imshow(lab_l_binary, cmap='gray')
#     ax4.set_title('lab_l_binary', fontsize=40)
    
#     ax5.imshow(lab_a_binary, cmap='gray')
#     ax5.set_title('lab_a_binary', fontsize=40)    
    
#     ax6.imshow(combined, cmap='gray')
#     ax6.set_title('combined', fontsize=40)    
    
#     plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## 4. Perspective transform (bird-eye view)
* perspective transform image as ROI
* Before perpective transforming, image is undistorted and binarized
* Input images PATH = /test_images
* Output Images PATH = /output_images/04_perspective_transform

In [23]:
# Load calibration parameter(mtx, dist) from pickle file
dist_pickle = pickle.load( open( "calibration.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

# Load imgae file using glob()
images = glob.glob('test_images/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_size = (img.shape[1], img.shape[0])
    
    # Distortion Correction to 'img'
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    # To take binary images, I use binary_threshold() that was written above section.
    binary_img = binary_threshold(undist)
    
    # For perspective transform, I add the warpImage() function
    src, warped_img = warpImage(undist, img_size)

    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(undist)
    ax1.set_title('Image ROI', fontsize=40)
    x = [src[0][0],src[1][0],src[2][0],src[3][0],src[0][0]]
    y = [src[0][1],src[1][1],src[2][1],src[3][1],src[0][1]]
    ax1.plot(x, y, color='#33cc99', alpha=0.4, linewidth=3, solid_capstyle='round', zorder=2)
    ax2.imshow(warped_img, cmap='gray')
    ax2.set_title('warped image', fontsize=40)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
    filename = os.path.basename(fname)
    
    # save the images
    plt.savefig('output_images/04_perspective_transform/'+filename)

## 5. Detect lane pixel and fitting curve
* Detect left and light lane line pixcels using histogram
* Fitting a polynomial to lane lines
* Input images PATH = /test_images
* Output Images PATH = /output_images/05_pixelandfitting

In [24]:
images = glob.glob('test_images/*.jpg')

# Load calibration parameter(mtx, dist) from pickle file
dist_pickle = pickle.load( open( "calibration.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]


for fname in images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_size = (img.shape[1], img.shape[0])
    
    # Distortion Correction to 'img'
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    # To take binary images, I use binary_threshold() that was written above section.
    binary_img = binary_threshold(undist)
    
    # For perspective transform, I add the warpImage() function
    src, warped_img = warpImage(binary_img, img_size)
    
    left_fit, right_fit, left_fitx, right_fitx, ploty, out_img = fit_polynomial(warped_img)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(warped_img, cmap ='gray')
    ax1.set_title('warped bin image', fontsize=40)
    ax2.imshow(out_img)
    ax2.plot(left_fitx, ploty, color='yellow')
    ax2.plot(right_fitx, ploty, color='yellow')
    ax2.set_title('lane pixels and fitting', fontsize=40)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
    filename = os.path.basename(fname)
    
    # save the images
    plt.savefig('output_images/05_detect_and_fitting/'+filename)

## 6 & 7 & 8. Curvature, Offset and Unwarp
* Calculate curvature and offset of the lanes
* Unwarp processed image to original perspective.
* Input images PATH = /test_images
* Output Images PATH = /output_images/06_07_complete_images

In [47]:
images = glob.glob('test_images/*.jpg')

# Load calibration parameter(mtx, dist) from pickle file
dist_pickle = pickle.load( open( "calibration.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]


for fname in images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    prior_find = False
    first_time = True
    
    # Call image process function
    # Init image -> Undistort -> Binary -> Warp -> Find lanes -> Unwarp
    result_img = complete_img(img, mtx, dist)
    
    plt.figure(figsize=(24, 9))
    plt.imshow(result_img)
    plt.title('compelete image', fontsize=40)
#     plt.text(50, 20,curv_str, fontsize = 20, color='white', 
#              horizontalalignment='left', verticalalignment='top')
    
    filename = os.path.basename(fname)
    
    # save the images
    plt.savefig('output_images/06_07_complete_images/'+filename)

## 9. Apply to video

In [9]:
def video_img_process(img):
   
    # Load calibration parameter(mtx, dist) from pickle file
    dist_pickle = pickle.load( open( "calibration.p", "rb" ) )
    mtx = dist_pickle["mtx"]
    dist = dist_pickle["dist"]
    
    result_img = complete_img(img, mtx, dist)
        
    return result_img


In [11]:
video_output = 'output_videos/project_video.mp4'

prior_find = False
first_time = True

clip1 = VideoFileClip('project_video.mp4') #.subclip(0,5)
video_clip = clip1.fl_image(video_img_process)
%time video_clip.write_videofile(video_output, audio=False)

                                                             

                                                       
t:  33%|███▎      | 41/125 [01:29<00:18,  4.54it/s, now=None]

t:  53%|█████▎    | 66/125 [00:22<00:13,  4.52it/s, now=None]
                                                             

                                                       
t:  33%|███▎      | 41/125 [01:29<00:18,  4.54it/s, now=None]

t:  54%|█████▎    | 67/125 [00:47<00:13,  4.38it/s, now=None]


t:   0%|          | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_videos/project_video.mp4.
Moviepy - Writing video output_videos/project_video.mp4






t:   0%|          | 2/1260 [00:00<02:54,  7.22it/s, now=None]


t:   0%|          | 3/1260 [00:00<03:41,  5.67it/s, now=None]


t:   0%|          | 4/1260 [00:00<04:05,  5.12it/s, now=None]


t:   0%|          | 5/1260 [00:01<04:17,  4.88it/s, now=None]


t:   0%|          | 6/1260 [00:01<04:24,  4.74it/s, now=None]


t:   1%|          | 7/1260 [00:01<04:29,  4.64it/s, now=None]


t:   1%|          | 8/1260 [00:01<04:36,  4.53it/s, now=None]


t:   1%|          | 9/1260 [00:01<04:49,  4.31it/s, now=None]


t:   1%|          | 10/1260 [00:02<04:47,  4.36it/s, now=None]


t:   1%|          | 11/1260 [00:02<04:47,  4.34it/s, now=None]


t:   1%|          | 12/1260 [00:02<04:57,  4.20it/s, now=None]


t:   1%|          | 13/1260 [00:02<05:04,  4.09it/s, now=None]


t:   1%|          | 14/1260 [00:03<05:21,  3.87it/s, now=None]


t:   1%|          | 15/1260 [00:03<05:26,  3.81it/s, now=None]


t:   1%|▏         | 16/1260 [00:03<05:24,  3.83it/s, now=None]


t:   1%|▏         | 17/1260 [0

t:   9%|▉         | 112/1260 [00:26<04:44,  4.03it/s, now=None]


t:   9%|▉         | 113/1260 [00:26<04:39,  4.10it/s, now=None]


t:   9%|▉         | 114/1260 [00:26<04:30,  4.23it/s, now=None]


t:   9%|▉         | 115/1260 [00:26<04:29,  4.25it/s, now=None]


t:   9%|▉         | 116/1260 [00:27<04:22,  4.36it/s, now=None]


t:   9%|▉         | 117/1260 [00:27<04:15,  4.48it/s, now=None]


t:   9%|▉         | 118/1260 [00:27<04:14,  4.49it/s, now=None]


t:   9%|▉         | 119/1260 [00:27<04:11,  4.53it/s, now=None]


t:  10%|▉         | 120/1260 [00:27<04:11,  4.54it/s, now=None]


t:  10%|▉         | 121/1260 [00:28<04:19,  4.39it/s, now=None]


t:  10%|▉         | 122/1260 [00:28<04:19,  4.39it/s, now=None]


t:  10%|▉         | 123/1260 [00:28<04:16,  4.43it/s, now=None]


t:  10%|▉         | 124/1260 [00:28<04:11,  4.52it/s, now=None]


t:  10%|▉         | 125/1260 [00:29<04:08,  4.57it/s, now=None]


t:  10%|█         | 126/1260 [00:29<04:12,  4.48it/s, now=None]


t:  10%|█ 

t:  18%|█▊        | 221/1260 [00:49<03:47,  4.57it/s, now=None]


t:  18%|█▊        | 222/1260 [00:50<03:46,  4.58it/s, now=None]


t:  18%|█▊        | 223/1260 [00:50<03:46,  4.57it/s, now=None]


t:  18%|█▊        | 224/1260 [00:50<03:44,  4.62it/s, now=None]


t:  18%|█▊        | 225/1260 [00:50<03:44,  4.60it/s, now=None]


t:  18%|█▊        | 226/1260 [00:51<03:49,  4.50it/s, now=None]


t:  18%|█▊        | 227/1260 [00:51<03:48,  4.52it/s, now=None]


t:  18%|█▊        | 228/1260 [00:51<03:44,  4.60it/s, now=None]


t:  18%|█▊        | 229/1260 [00:51<03:42,  4.63it/s, now=None]


t:  18%|█▊        | 230/1260 [00:51<03:42,  4.64it/s, now=None]


t:  18%|█▊        | 231/1260 [00:52<03:43,  4.61it/s, now=None]


t:  18%|█▊        | 232/1260 [00:52<03:40,  4.66it/s, now=None]


t:  18%|█▊        | 233/1260 [00:52<03:41,  4.64it/s, now=None]


t:  19%|█▊        | 234/1260 [00:52<03:39,  4.66it/s, now=None]


t:  19%|█▊        | 235/1260 [00:52<03:38,  4.69it/s, now=None]


t:  19%|█▊

...

t:  69%|██████▉   | 875/1260 [03:12<01:25,  4.53it/s, now=None]


t:  70%|██████▉   | 876/1260 [03:13<01:24,  4.57it/s, now=None]


t:  70%|██████▉   | 877/1260 [03:13<01:23,  4.59it/s, now=None]


t:  70%|██████▉   | 878/1260 [03:13<01:22,  4.62it/s, now=None]


t:  70%|██████▉   | 879/1260 [03:13<01:25,  4.47it/s, now=None]


t:  70%|██████▉   | 880/1260 [03:14<01:25,  4.45it/s, now=None]


t:  70%|██████▉   | 881/1260 [03:14<01:23,  4.53it/s, now=None]


t:  70%|███████   | 882/1260 [03:14<01:22,  4.56it/s, now=None]


t:  70%|███████   | 883/1260 [03:14<01:22,  4.57it/s, now=None]


t:  70%|███████   | 884/1260 [03:14<01:21,  4.59it/s, now=None]


t:  70%|███████   | 885/1260 [03:15<01:21,  4.62it/s, now=None]


t:  70%|███████   | 886/1260 [03:15<01:21,  4.61it/s, now=None]


t:  70%|███████   | 887/1260 [03:15<01:21,  4.59it/s, now=None]


t:  70%|███████   | 888/1260 [03:15<01:20,  4.61it/s, now=None]


t:  71%|███████   | 889/1260 [03:16<01:20,  4.63it/s, now=None]


t:  71%|██

t:  78%|███████▊  | 984/1260 [03:36<01:01,  4.51it/s, now=None]


t:  78%|███████▊  | 985/1260 [03:37<01:00,  4.54it/s, now=None]


t:  78%|███████▊  | 986/1260 [03:37<00:59,  4.61it/s, now=None]


t:  78%|███████▊  | 987/1260 [03:37<00:59,  4.58it/s, now=None]


t:  78%|███████▊  | 988/1260 [03:37<00:58,  4.63it/s, now=None]


t:  78%|███████▊  | 989/1260 [03:37<00:58,  4.66it/s, now=None]


t:  79%|███████▊  | 990/1260 [03:38<00:58,  4.63it/s, now=None]


t:  79%|███████▊  | 991/1260 [03:38<00:58,  4.60it/s, now=None]


t:  79%|███████▊  | 992/1260 [03:38<00:57,  4.63it/s, now=None]


t:  79%|███████▉  | 993/1260 [03:38<00:57,  4.63it/s, now=None]


t:  79%|███████▉  | 994/1260 [03:39<00:57,  4.64it/s, now=None]


t:  79%|███████▉  | 995/1260 [03:39<00:57,  4.63it/s, now=None]


t:  79%|███████▉  | 996/1260 [03:39<00:57,  4.61it/s, now=None]


t:  79%|███████▉  | 997/1260 [03:39<00:56,  4.68it/s, now=None]


t:  79%|███████▉  | 998/1260 [03:39<00:56,  4.65it/s, now=None]


t:  79%|██

t:  87%|████████▋ | 1092/1260 [04:00<00:36,  4.62it/s, now=None]


t:  87%|████████▋ | 1093/1260 [04:00<00:36,  4.58it/s, now=None]


t:  87%|████████▋ | 1094/1260 [04:01<00:36,  4.60it/s, now=None]


t:  87%|████████▋ | 1095/1260 [04:01<00:35,  4.63it/s, now=None]


t:  87%|████████▋ | 1096/1260 [04:01<00:37,  4.42it/s, now=None]


t:  87%|████████▋ | 1097/1260 [04:01<00:36,  4.44it/s, now=None]


t:  87%|████████▋ | 1098/1260 [04:02<00:37,  4.37it/s, now=None]


t:  87%|████████▋ | 1099/1260 [04:02<00:36,  4.38it/s, now=None]


t:  87%|████████▋ | 1100/1260 [04:02<00:35,  4.51it/s, now=None]


t:  87%|████████▋ | 1101/1260 [04:02<00:34,  4.59it/s, now=None]


t:  87%|████████▋ | 1102/1260 [04:02<00:34,  4.63it/s, now=None]


t:  88%|████████▊ | 1103/1260 [04:03<00:33,  4.64it/s, now=None]


t:  88%|████████▊ | 1104/1260 [04:03<00:33,  4.61it/s, now=None]


t:  88%|████████▊ | 1105/1260 [04:03<00:33,  4.63it/s, now=None]


t:  88%|████████▊ | 1106/1260 [04:03<00:33,  4.67it/s, now=Non

t:  95%|█████████▌| 1199/1260 [04:24<00:13,  4.56it/s, now=None]


t:  95%|█████████▌| 1200/1260 [04:24<00:13,  4.58it/s, now=None]


t:  95%|█████████▌| 1201/1260 [04:24<00:12,  4.63it/s, now=None]


t:  95%|█████████▌| 1202/1260 [04:24<00:12,  4.58it/s, now=None]


t:  95%|█████████▌| 1203/1260 [04:25<00:12,  4.59it/s, now=None]


t:  96%|█████████▌| 1204/1260 [04:25<00:12,  4.57it/s, now=None]


t:  96%|█████████▌| 1205/1260 [04:25<00:11,  4.59it/s, now=None]


t:  96%|█████████▌| 1206/1260 [04:25<00:11,  4.62it/s, now=None]


t:  96%|█████████▌| 1207/1260 [04:26<00:11,  4.62it/s, now=None]


t:  96%|█████████▌| 1208/1260 [04:26<00:11,  4.66it/s, now=None]


t:  96%|█████████▌| 1209/1260 [04:26<00:10,  4.66it/s, now=None]


t:  96%|█████████▌| 1210/1260 [04:26<00:10,  4.68it/s, now=None]


t:  96%|█████████▌| 1211/1260 [04:26<00:10,  4.60it/s, now=None]


t:  96%|█████████▌| 1212/1260 [04:27<00:10,  4.53it/s, now=None]


t:  96%|█████████▋| 1213/1260 [04:27<00:10,  4.57it/s, now=Non

Moviepy - Done !
Moviepy - video ready output_videos/project_video.mp4
CPU times: user 7min 56s, sys: 1min 8s, total: 9min 5s
Wall time: 4min 38s


In [17]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))

## Challenge Video
* Fail.....

In [1]:
# video_output = 'output_videos/challenge_video.mp4'

# prior_find = False
# first_time = True

# clip1 = VideoFileClip('challenge_video.mp4').subclip(0,5)
# video_clip = clip1.fl_image(video_img_process)
# %time video_clip.write_videofile(video_output, audio=False)

In [13]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))

## challenge video trouble shooting
* Fail....

In [102]:
# img = cv2.imread('error_image.jpg')
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# img_size = (img.shape[1], img.shape[0])

# # Distortion Correction to 'img'
# undist = cv2.undistort(img, mtx, dist, None, mtx)
# # To take binary images, I use binary_threshold() that was written above section.
# ksize = 15

# # Take gardient threshold value
# grad_x_binary = abs_sobel_thresh(img, orient='x', sobel_kernel=ksize, thresh=(20, 255))
# grad_y_binary = abs_sobel_thresh(img, orient='y', sobel_kernel=ksize, thresh=(20, 255))
# mag_binary = mag_thresh(img, sobel_kernel=ksize, mag_thresh=(30, 100))
# dir_binary = dir_threshold(img, sobel_kernel=ksize, thresh=(0.7, 1.3))

# # Take color threshold value
# hls_h_binary, hls_l_binary, hls_s_binary = hls_color_threshold(img,(130,255),(100,255),(170,255))
# hsv_h_binary, hsv_s_binary, hsv_v_binary = hsv_color_threshold(img,(130,255),(130,255),(170,255))
# lab_l_binary, lab_a_binary, lab_b_binary = lab_color_threshold(img,(130,255),(130,255),(130,255))

# combined = np.zeros_like(hls_s_binary)

# # ('sobel x' or 'hls_s_bin') and 'hsv_v_bin' <- Overlap condition
# combined[((grad_x_binary == 1) | (hls_s_binary == 1)) & (hsv_v_binary == 1)] = 1

# # For perspective transform, I add the warpImage() function
# src, warped_img = warpImage(binary_img, img_size)


# # Plot the result
# f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 9))
# f.tight_layout()

# ax1.imshow(grad_x_binary, cmap='gray')
# ax1.set_title('grad_x_binary', fontsize=40)

# ax2.imshow(hls_s_binary, cmap='gray')
# ax2.set_title('hls_s_binary', fontsize=40)

# ax3.imshow(hsv_v_binary, cmap='gray')
# ax3.set_title('hsv_v_binary', fontsize=40)    

# plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
